![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [100]:
import pandas as pd
import numpy as np

# Start coding here...

In [101]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [102]:
# subsetting df into client, campaign and economics DataFrames
# df.info()
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
# client.info()

# cleaning the client['job'] column
client['job'] = client['job'].str.replace(".", "_")
# print(client['job'])
# print(client['education'])

# cleaning the client['education'] column
client['education'] = client['education'].str.replace(".", "_")
client['education'] = client['education'].replace("unknown", pd.NA)
# print(client['education'])

# cleaning the client['credit_default'] column
map_cred = {"yes": 1, "no": 0, "unknown": 0}
client['credit_default'] = client['credit_default'].map(map_cred)
client['credit_default'] = client['credit_default'].astype('bool')
#client['credit_default'].info()

# cleaning the client['mortgage'] column
map_mort = {"yes": 1, "no": 0, "unknown": 0}
client['mortgage'] = client['mortgage'].map(map_mort)
client['mortgage'] = client['mortgage'].astype('bool')
client['mortgage'].info()

campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
#campaign.info()

# cleaning the campaign['previous_outcome'] column
map_prev = {"success": 1, "failure": 0, "nonexistent": 0}
campaign['previous_outcome'] = campaign['previous_outcome'].map(map_prev)
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')
campaign['previous_outcome'].info()

# cleaning the campaign['campaign_outcome'] column
map_camp = {"yes": 1, "no": 0}
campaign['campaign_outcome'] = campaign['campaign_outcome'].map(map_camp)
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')
#campaign['campaign_outcome'].info()

df['year'] = '2022'
df['year'] = df['year'].astype('int')
df['month'] = df['month'].astype('string')
df.info()

# Fix: Convert month names to numbers before creating the date
# Create a mapping from month names to numbers
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
df['month'] = df['month'].str.lower()
df['month'] = df['month'].map(month_map)

campaign['last_contact_date'] = pd.to_datetime(
    df[['year', 'month','day']]
)
print(type(campaign['last_contact_date']))
print(campaign['last_contact_date'].value_counts())
campaign.info()
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
# economics.info()
client.to_csv('client.csv', index = False)
campaign.to_csv('campaign.csv', index = False)
economics.to_csv('economics.csv', index = False)
dd = pd.read_csv('client.csv')
print(dd.head())

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: mortgage
Non-Null Count  Dtype
--------------  -----
41188 non-null  bool 
dtypes: bool(1)
memory usage: 40.3 KB
<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: previous_outcome
Non-Null Count  Dtype
--------------  -----
41188 non-null  bool 
dtypes: bool(1)
memory usage: 40.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage              